In [2]:
import time
import json
import random
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from webdriver_manager.microsoft import EdgeChromiumDriverManager
def process_review_count(text):
    text = text.strip().replace(',', '')
    if 'K+' in text:
        return str(int(float(text.replace('(', '').replace(')', '').replace('K+', '').strip()) * 1000))
    return text
def setup_driver():
    options = webdriver.EdgeOptions()
    # options.add_argument('--headless')  # Run browser in headless mode
    options.add_argument('--no-sandbox')
    try:
        driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()), options=options)
    except Exception as e:
        print(e)
        raise Exception("Failed to install Edge Chromium driver.")
    return driver
def scrape_amazon(categories):
    driver = setup_driver()
    all_products = []
    seen_products = set()  # Initialize the set here
    for category, base_url in categories.items():
        products = []
        for page in range(1, 2):
            url = f"{base_url}&page={page}"
            try:
                driver.get(url)
                WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".s-result-item")))
            except TimeoutException:
                print(f"Timed out waiting for elements on page {page} of category {category}.")
                continue
            # Use random sleep to mimic human behavior
            time.sleep(random.uniform(3.0, 6.0))
            # Now parse the page with BeautifulSoup
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            for product in soup.find_all('div', class_='sg-col-inner'):
                product_dict = {}
                # Extract Product_ID (ASIN)
                product_uuid_div = product.find('div', attrs={"data-uuid": True})
                if product_uuid_div:
                    product_dict['Product_ID'] = product_uuid_div['data-uuid']
                # Item name
                item_name = product.find('span', class_='a-size-base-plus a-color-base a-text-normal')
                if not item_name:
                    item_name = product.find('span', class_='a-size-medium a-color-base a-text-normal')
                if item_name:
                    product_dict['product'] = item_name.text.strip()
                # Item price
                product_price = product.find('span', class_='a-offscreen')
                if product_price:
                    product_price = product_price.text.strip().replace("$", "").replace(",", "").strip()
                    product_dict['price'] = product_price
                # Ratings and review responders
                rating_spans = product.find_all('span', attrs={"aria-label": True})
                for rating_span in rating_spans:
                    aria_label_value = rating_span.attrs["aria-label"]
                    if "stars" in aria_label_value:
                        product_dict['ratings'] = aria_label_value.split(" ")[0]
                    else:
                        if 'K+' in aria_label_value:
                            product_dict['review_responders'] = aria_label_value
                        else:
                            try:
                                int_value = int(aria_label_value)
                                product_dict['review_responders'] = aria_label_value
                            except ValueError:
                                pass
                # Item reviews
                item_reviews = product.find('span', class_='a-size-base s-underline-text')
                if item_reviews:
                    reviews_text = item_reviews.text.strip()
                    reviews_count = process_review_count(reviews_text)
                    product_dict['reviews'] = reviews_count
                # Extract item URL
                item_url_tag = product.find('a', class_='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')
                if item_url_tag:
                    item_url = item_url_tag.get('href')
                    product_dict['url'] = "https://www.amazon.com" + item_url
                else:
                    product_dict['url'] = None
                # Add category to product_dict
                product_dict['category'] = category
                # Ensure product dictionary contains necessary details
                if 'Product_ID' in product_dict and product_dict['Product_ID']:
                    # Create a unique identifier for the product
                    identifier = product_dict['Product_ID']
                    if identifier not in seen_products:
                        seen_products.add(identifier)
                        products.append(product_dict)
            all_products.extend(products)
    driver.quit()
    return json.dumps(all_products)
if __name__ == '__main__':
    categories = {
        'Smartphones': 'https://www.amazon.com/s?k=smartphone&ref=nb_sb_noss',
        'Laptops': 'https://www.amazon.com/s?k=Laptops&ref=nb_sb_noss',
        'video_games': 'https://www.amazon.com/s?k=video_games&ref=nb_sb_noss',
        'Dresses':'https://www.amazon.com/s?k=Dresses&ref=nb_sb_noss',
        'Shoes':'https://www.amazon.com/s?k=Shoes&ref=nb_sb_noss',
        'Accessories':'https://www.amazon.com/s?k=accessories+for+clothes&ref=nb_sb_noss',
    }
    amazon_data = json.loads(scrape_amazon(categories))
    # Save the JSON data to a file
    with open('amazon_data_cat.json', 'w') as file:
        json.dump(amazon_data, file)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: MicrosoftEdge=117.0.2045.31)
Stacktrace:
	GetHandleVerifier [0x00007FF6BCA999D2+63682]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00007FF6BCA21922+267714]
	(No symbol) [0x00007FF6BC7DAC0E]
	(No symbol) [0x00007FF6BC7B28C7]
	(No symbol) [0x00007FF6BC850B9F]
	(No symbol) [0x00007FF6BC8663E8]
	(No symbol) [0x00007FF6BC84C623]
	(No symbol) [0x00007FF6BC81DA7A]
	(No symbol) [0x00007FF6BC81CD6B]
	(No symbol) [0x00007FF6BC81E204]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF6BCC667A9+1233737]
	(No symbol) [0x00007FF6BC89B274]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF6BC9632AA+33498]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF6BC95BDE9+3609]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF6BCC65544+1229028]
	Microsoft::Applications::

In [4]:
import pandas as pd 
df = pd.read_csv('amazon_data-5.csv')
df

,Product_ID,product,price,ratings,reviews,url,category
0,e811560b-3016-4dfa-b676-c05c5e879aa1,Google Pixel 7a - Unlocked Android Cell Phone ...,449.00,4.7,3041,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,Smartphones
1,b3b23cef-6675-4d35-9500-a18615404f52,Google Pixel 7a - Unlocked Android Cell Phone ...,444.00,4.5,516,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,Smartphones
2,f4c9ccf8-d56f-43d6-b768-5646d6c135b0,Samsung Electronics Galaxy Note 20 Ultra 5G Fa...,368.00,4.1,3220,https://www.amazon.com/Samsung-Electronics-Fac...,Smartphones
3,d3b46b5c-9fb9-4f8e-8862-279b02a8b71a,"SAMSUNG Galaxy S23 Ultra Cell Phone, Factory U...",999.99,4.7,377,https://www.amazon.com/SAMSUNG-Factory-Unlocke...,Smartphones
4,46a10a36-a2c1-4b7d-ba82-15e14e319fe7,"SAMSUNG Galaxy S23 Ultra Cell Phone, Factory U...",149.99,4.5,10,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,Smartphones
...,...,...,...,...,...,...,...
534,0a6b9ce0-179e-42b2-a4ad-a546a4830601,"Grisofaa Clothes Organizer for Folded Clothes,...",49.99,4.5,84,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,Accessories
535,9366c64b-30a0-4497-ab26-4d0badb95d91,ULENDIS 6 Pack Upgraded Washable Wardrobe Clot...,25.99,4.0,3,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,Accessories
536,feca223c-f7e9-46d6-be9f-65874d308fc3,ULENDIS 9 Pack Upgraded Wardrobe Clothes Organ...,36.99,4.5,3,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,Accessories
537,07f2589c-7dc7-4a94-b586-8f583aed5c4a,Womens Skinny Stretch Cinch Belt Elastic Waist...,11.99,5.0,452,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,Accessories


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 539 entries, 0 to 538
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Product_ID  539 non-null    object 
 1   product     539 non-null    object 
 2   price       539 non-null    float64
 3   ratings     539 non-null    float64
 4   reviews     539 non-null    int64  
 5   url         539 non-null    object 
 6   category    539 non-null    object 
dtypes: float64(2), int64(1), object(4)
memory usage: 29.6+ KB


In [13]:
df[df.reviews.duplicated()]

,Product_ID,product,price,ratings,reviews,url,category
8,8601abc4-1f34-465c-8f55-a29aaf6ae1ef,UMIDIGI A15C (16+128GB) Android 13 NFC Unlocke...,149.99,4.5,10,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,Smartphones
14,2e8c04d2-0d2e-4890-932b-cffd84d70157,"UMIDIGI Unlocked Cell Phone G3 Plus,Android 13...",103.99,3.6,231,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,Smartphones
26,c26b724c-50e3-4501-b136-e250a042f9de,UMIDIGI A15C (16+128GB) Android 13 NFC Unlocke...,149.99,4.5,10,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,Smartphones
27,9120b64f-7f51-4fa3-a037-15de961b2c82,"SAMSUNG Galaxy S23 Ultra Cell Phone, Factory U...",149.99,3.6,10,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,Smartphones
29,c2524e42-c98c-4b7c-96d0-b14f194165d0,UMIDIGI A15C (16+128GB) Android 13 NFC Unlocke...,149.99,4.7,10,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,Smartphones
...,...,...,...,...,...,...,...
532,d127e358-26ce-4d6b-b268-0f6e2de444f9,"Trrcylp 2Pack Pants Hangers Space Saving, 9 La...",12.99,4.3,27,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,Accessories
534,0a6b9ce0-179e-42b2-a4ad-a546a4830601,"Grisofaa Clothes Organizer for Folded Clothes,...",49.99,4.5,84,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,Accessories
535,9366c64b-30a0-4497-ab26-4d0badb95d91,ULENDIS 6 Pack Upgraded Washable Wardrobe Clot...,25.99,4.0,3,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,Accessories
536,feca223c-f7e9-46d6-be9f-65874d308fc3,ULENDIS 9 Pack Upgraded Wardrobe Clothes Organ...,36.99,4.5,3,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,Accessories


In [11]:
df.reviews.duplicated().sum()

283

In [ ]:
df

[0      https://www.amazon.com/sspa/click?ie=UTF8&spc=...
 1      https://www.amazon.com/sspa/click?ie=UTF8&spc=...
 2      https://www.amazon.com/Samsung-Electronics-Fac...
 3      https://www.amazon.com/SAMSUNG-Factory-Unlocke...
 4      https://www.amazon.com/sspa/click?ie=UTF8&spc=...
                              ...                        
 534    https://www.amazon.com/sspa/click?ie=UTF8&spc=...
 535    https://www.amazon.com/sspa/click?ie=UTF8&spc=...
 536    https://www.amazon.com/sspa/click?ie=UTF8&spc=...
 537    https://www.amazon.com/sspa/click?ie=UTF8&spc=...
 538    https://www.amazon.com/sspa/click?ie=UTF8&spc=...
 Name: url, Length: 539, dtype: object]